In [ ]:
import pandas as pd
import hvplot.pandas
import panel as pn
from datetime import date, datetime, timedelta
from infra_copel import MongoHistoricoOficial
from bokeh.models.formatters import DatetimeTickFormatter

mongo = MongoHistoricoOficial()
data_pesquisa_inicial = date.today() - timedelta(days=7)


def filtrar_data(data: date):
    resultado = mongo["ons_geracao_usinas"].find(
        {
            "usina": "Itaipu",
            "periodo": {"$gte": datetime(data.year, data.month, data.day)},
        }
    )
    lista = list(resultado)
    df = pd.DataFrame(lista)
    print(f'Uso memória: {df.memory_usage().sum() / 1024**2} MB')

    return df.hvplot(
        x="periodo", y="geracao_mwm", xformatter=DatetimeTickFormatter(days="%d/%m/%Y")
    )


filtro_data = pn.widgets.DatePicker(name="Data", value=data_pesquisa_inicial)
# filtro_usina = pn.widgets.Select(name="Usina", options=lista_usinas())

filtered_view = pn.Column(
    pn.Row(filtro_data),
    pn.panel(
        pn.bind(filtrar_data, data=filtro_data),
        width=800,
        height=600,
    ),
)

filtered_view